# Testing One-Hot Encoding with MasterData5

Our original "Testing One-Hot Encoding" workbook used MasterData3 and looked at encoding for our multi-class models. Here we'll use MasterData5 and look at encoding for our binary models, with Completion Status as our target variable instead of familyStatus.

## Set Up Libraries and Data

In [24]:
# Import necessary data libraries.
import pandas as pd
import os 
import csv
import io
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

In [25]:
# Set up URLs.
circuits_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/circuits.csv'
constructor_results_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/constructor_results.csv'
constructor_standings_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/constructor_standings.csv'
constructors_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/constructors.csv'
driver_standings_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/driver_standings.csv'
drivers_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/drivers.csv'
lap_times_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/lap_times.csv'
pit_stop_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/pit_stops.csv'
qualifying_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/qualifying.csv'
races_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/races.csv'
results_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/results.csv'
seasons_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/seasons.csv'
status_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/status.csv'
race_status_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/race_status.csv'
master_data_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/processed/MasterData5.csv'

In [26]:
# Set up dataframes.
circuits_df = pd.read_csv(circuits_url, sep = ',', encoding = 'latin-1')
constructor_results_df = pd.read_csv(constructor_results_url, sep = ',', engine = 'python')
constructor_standings_df = pd.read_csv(constructor_standings_url, sep = ',', engine = 'python')
constructors_df = pd.read_csv(constructor_standings_url, sep = ',', engine = 'python')
driver_standings_df = pd.read_csv(driver_standings_url, sep = ',', engine = 'python')
lap_times_df = pd.read_csv(lap_times_url, sep = ',', engine = 'python')
pit_stop_df = pd.read_csv(pit_stop_url, sep = ',', engine = 'python')
qualifying_df = pd.read_csv(constructor_standings_url, sep = ',', engine = 'python')
results_df = pd.read_csv(results_url, sep = ',', engine = 'python')
seasons_df = pd.read_csv(seasons_url, sep = ',', engine = 'python')
status_df = pd.read_csv(status_url, sep = ',', engine = 'python')
races_df = pd.read_csv(races_url, sep = ',', engine = 'c')
drivers_df = pd.read_csv(drivers_url, sep = ',', encoding = 'latin-1')
race_status_df = pd.read_csv(race_status_url, sep = ',', engine = 'python')
master_data = pd.read_csv(master_data_url, sep = ',', engine = 'python')

## Establishing Variables

In [27]:
# Are there any NAs in our data?
master_data.isna().sum()

raceId                  0
driverId                0
constructorId           0
grid                    0
position             2164
positionText            0
positionOrder           0
laps                    0
fastestLap           2745
rank                 2704
fastestLapSpeed      2745
familyStatus            0
Completion Status       0
year                    0
circuitId               0
country                 0
alt                     0
isHistoric              0
trackType               0
nationality             0
total_lap_time          0
average_lap_time        0
minimum_lap_time        0
PRCP                    0
TAVG                    0
TMAX                    0
TMIN                    0
bundled_circuitId       0
binned_circuits         0
dtype: int64

Yes, we have NAs in position, fastestLap, rank, and fastestLapSpeed. We currently do not plan on using these in our models anyway.

In [28]:
# Check on the mins and maxes of Completion Status.
master_data["Completion Status"].describe()

count    9466.000000
mean        0.750898
std         0.432516
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Completion Status, dtype: float64

We can see above that our minimum Completion Status is 0 and our maximum Completion Status is 1. Since Completion Status is a binary variable, this makes sense. We can also see, however, that the mean is 0.751, and as low as our 25 percentile mark is 1. This indicates a class imbalance.

## Start One-Hot Encoding

We used these sites (https://towardsdatascience.com/target-encoding-for-multi-class-classification-c9a7bcb1a53 and https://www.analyticsvidhya.com/blog/2021/05/how-to-perform-one-hot-encoding-for-multi-categorical-variables/) as foundations for our code.

In [29]:
# Rename Completion Status so it doesn't have any spaces. This will make it easier to use in the code below.
master_data = master_data.rename(columns={"Completion Status": "CompletionStatus"})

In [30]:
# Make track type a string feature instead of numeric.
master_data["trackType"] = master_data.trackType.astype("str")

In [31]:
# Analytics Vidhya suggests we check features before proceeding. Do so using their code.
check_features = master_data.select_dtypes(include='O').keys()
# Display variables.
check_features

Index(['positionText', 'country', 'trackType', 'nationality',
       'bundled_circuitId', 'binned_circuits'],
      dtype='object')

In [32]:
"""
Encode and transform CompletionStatus using the one-hot encoding code from Towards Data Science.
Note that the target variable must be a string here.
"""
encodeFamilyStatus = ce.OneHotEncoder().fit(master_data.CompletionStatus.astype(str))
y_onehot = encodeFamilyStatus.transform(master_data.CompletionStatus.astype(str))
y_onehot

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,CompletionStatus_1,CompletionStatus_2
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
...,...,...
9461,0,1
9462,0,1
9463,0,1
9464,0,1


In [33]:
"""
One-hot encode the country column using the for loop shown in the Towards Data Science article.
"""
class_names = y_onehot.columns
for class_ in class_names:
  encodeCountry = ce.TargetEncoder(smoothing = 0)
  print(encodeCountry.fit_transform(master_data["country"], y_onehot[class_]))

       country
0     0.351812
1     0.351812
2     0.351812
3     0.351812
4     0.351812
...        ...
9461  0.271930
9462  0.271930
9463  0.271930
9464  0.271930
9465  0.271930

[9466 rows x 1 columns]
       country
0     0.648188
1     0.648188
2     0.648188
3     0.648188
4     0.648188
...        ...
9461  0.728070
9462  0.728070
9463  0.728070
9464  0.728070
9465  0.728070

[9466 rows x 1 columns]


/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [34]:
"""
The Towards Data Science article gives a function, target_encode_multiclass,
in which one-hot encodes the entire dataset with the given target variable. That function is below.
"""
def target_encode_multiclass(X,y): #X,y are pandas df and series
    y = y.astype(str)   #convert to string to onehot encode
    enc = ce.OneHotEncoder().fit(y)
    y_onehot = enc.transform(y)
    class_names = y_onehot.columns  #names of onehot encoded columns
    X_obj = X.select_dtypes('object') #separate categorical columns
    X = X.select_dtypes(exclude='object') 
    for class_ in class_names:
      
        enc = ce.TargetEncoder()
        enc.fit(X_obj,y_onehot[class_]) #convert all categorical 
        temp = enc.transform(X_obj)       #columns for class_
        temp.columns = [str(x)+'_'+str(class_) for x in temp.columns]
        X = pd.concat([X,temp],axis=1)    #add to original dataset
      
    return X

In [35]:
# Use the above function to one-hot encode our master_data dataset with CompletionStatus as our target variable.
onehot_data = target_encode_multiclass(master_data, master_data["CompletionStatus"])

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [36]:
# Take a look at the new dataset using the describe() function.
onehot_data.describe()

,raceId,driverId,constructorId,grid,position,positionOrder,laps,fastestLap,rank,fastestLapSpeed,...,trackType_CompletionStatus_1,nationality_CompletionStatus_1,bundled_circuitId_CompletionStatus_1,binned_circuits_CompletionStatus_1,positionText_CompletionStatus_2,country_CompletionStatus_2,trackType_CompletionStatus_2,nationality_CompletionStatus_2,bundled_circuitId_CompletionStatus_2,binned_circuits_CompletionStatus_2
count,9466.000000,9466.000000,9466.000000,9466.000000,7302.000000,9466.000000,9466.000000,6721.000000,6762.000000,6721.000000,...,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000
mean,500.169977,249.438411,36.706634,11.070357,8.744043,10.817135,52.982252,42.216039,10.692399,202.509826,...,0.249102,0.249099,0.249102,0.249102,0.750949,0.750898,0.750898,0.750901,0.750898,0.750898
std,408.988287,355.593273,63.937258,6.240870,5.090236,6.043638,17.737604,17.000168,6.059511,21.342117,...,0.021213,0.054796,0.017268,0.020310,0.407406,0.063220,0.021213,0.054796,0.017268,0.020310
min,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,0.000000,89.540000,...,0.237243,0.078947,0.159763,0.213611,0.000000,0.515625,0.712955,0.428572,0.703704,0.722412
25%,121.000000,15.000000,4.000000,6.000000,4.000000,6.000000,49.000000,32.000000,5.000000,192.346000,...,0.237243,0.209566,0.250732,0.235686,0.877551,0.720901,0.762757,0.707641,0.749268,0.746549
50%,236.000000,35.000000,9.000000,11.000000,8.000000,11.000000,56.000000,45.000000,11.000000,203.989000,...,0.237243,0.240838,0.250732,0.253451,0.968680,0.746711,0.762757,0.759162,0.749268,0.746549
75%,934.000000,810.000000,20.000000,16.000000,13.000000,16.000000,66.000000,54.000000,16.000000,215.688000,...,0.237243,0.292359,0.250732,0.253451,0.997835,0.780303,0.762757,0.790434,0.749268,0.764314
max,1060.000000,854.000000,214.000000,24.000000,24.000000,24.000000,87.000000,85.000000,24.000000,257.320000,...,0.287045,0.571428,0.296296,0.277588,1.000000,0.900000,0.762757,0.921053,0.840237,0.786389


### Create a CSV file with our new one-hot encoded dataset.

In [37]:
# Use pandas.DataFrame.to_csv to create the CSV file.
onehot_data.to_csv("data/processed/OneHot_MasterData5.csv", index = False)